In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import pandas as pd
import os

pdf = "Error_PDFS/ARP_09_18_2022.pdf"


In [2]:
#Testing for page
import header_scanner as hs

page_list = race_results_scanner.get_page_list(pdf)
target_page = 0

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
#result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
#print(result_tables[0])
#print(result_tables[1])

('allowance', {'type': 'allowance', 'word_list': [['track', '', '-', 1], ['date', '-', '-'], ['race_num', 'Race', 'ALLOWANCE'], ['criteria', 'FOR', '.', 2], ['track_length', '0', 'Current'], ['purse', 'Purse', '0', 2], ['weather', 'Weather', 'Track'], ['track_type', 'Track', 'Off'], ['off_time', 'at', 'Start'], ['start', 'Start', 'Timer']], 'split_word': 'Purse'})
('allowance claiming', {'type': 'allowance claiming', 'word_list': [['track', '', '-', 1], ['date', '-', '-'], ['race_num', 'Race', 'ALLOWANCE'], ['criteria', 'FOR', '.', 2], ['claiming_price', 'Price', '0', 2], ['track_length', '0', 'Current'], ['purse', 'Purse', '0', 2], ['weather', 'Weather', 'Track'], ['track_type', 'Track', 'Off'], ['off_time', 'at', 'Start'], ['start', 'Start', 'Timer']], 'split_word': 'Price'})
('claiming', {'type': 'claiming', 'word_list': [['track', '', '-', 1], ['date', '-', '-'], ['race_num', 'Race', 'CLAIMING'], ['criteria', 'FOR', '.', 2], ['claiming_price', 'Price', '0', 2], ['track_length', '0'

TypeError: unhashable type: 'dict'

In [4]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0]
    bottom_table = result_tables[1]
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
        #Merge into master pdf df
        if(len(pdf_df) < 1):
            pdf_df = merged_df
        else:
            pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
        print(pdf_df)
        #Merging into master 
        if(len(master_df) < 1):
            master_df = pdf_df
        else:
            master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 3}, {'page_num': 1, 'horse_count': 5}, {'page_num': 2, 'horse_count': 8}, {'page_num': 3, 'horse_count': 8}, {'page_num': 4, 'horse_count': 8}, {'page_num': 5, 'horse_count': 8}, {'page_num': 6, 'horse_count': 8}, {'page_num': 7, 'horse_count': 7}, {'page_num': 8, 'horse_count': 8}]
['ARAPAHOE', 'P', '*ARK', '-', 'September', '18,', '2022', '-', 'Race', '1', 'MAIDEN', 'SPECIAL', 'WEIGHT', '-', 'Arabian', 'FOR', 'MAIDENS,', 'FILLIES', 'AND', 'MARES', 'THREE', 'YEARS', 'OLD', 'AND', 'UPWARD.', 'Three', 'Year', 'Olds,', '120', 'lbs.;', 'Older,', '124', 'lbs.', 'Seven', 'Furlongs', 'On', 'The', 'Dirt', 'Current', 'Track', 'Record', '(Daring', 'Pegasus', '-', '1', '21.00', '-', 'July', '4,', '2003)', 'Purse', '$9,000', 'Available', 'Money', '$9,000', 'Value', 'of', 'Race', '$9,000', '1st', '$5,400,', '2nd', '$2,250,', '3rd', '$1,350', 'Wind', 'Speed', '0', 'Weather', 'Clear', 'Track', 'Fast', 'Off', 'at', '1', '01', 'Start', 'Good', 'for', 'all', 'Timer', 'El

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


    Last Raced  Pgm         Horse Name (Jockey)   PP Start 1/4  1/2  Str Fin  \
0  12Sep22 ARP  8.0  WMA GOT IT (Martin, David)  3.0   3.0   3  1.0  1.0   1   
1  12Sep22 ARP  5.0   SWEET-WAN (Herrera, Hugo)  1.0   2.0   2  2.0  2.0   2   
2  12Sep22 ARP  7.0     CIRCE JPH (Lopez, Jose)  2.0   1.0   1  3.0  3.0   3   

    Odds  ...          track                date race_num  \
0   1.00  ...  Arapahoe Park  September 18, 2022        1   
1  0.90*  ...  Arapahoe Park  September 18, 2022        1   
2   3.20  ...  Arapahoe Park  September 18, 2022        1   

                                            criteria  \
0  FOR MAIDENS, FILLIES AND MARES THREE YEARS OLD...   
1  FOR MAIDENS, FILLIES AND MARES THREE YEARS OLD...   
2  FOR MAIDENS, FILLIES AND MARES THREE YEARS OLD...   

                  track_length   purse weather track_type off_time  \
0   Seven Furlongs On The Dirt  $9,000   Clear       Fast     1:01   
1   Seven Furlongs On The Dirt  $9,000   Clear       Fast     1:01   

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a 

...

     Last Raced  Pgm             Horse Name (Jockey)   PP Start 1/4  1/2  Str  \
0   12Sep22 ARP  8.0      WMA GOT IT (Martin, David)  3.0   3.0   3  1.0  1.0   
1   12Sep22 ARP  5.0       SWEET-WAN (Herrera, Hugo)  1.0   2.0   2  2.0  2.0   
2   12Sep22 ARP  7.0         CIRCE JPH (Lopez, Jose)  2.0   1.0   1  3.0  3.0   
3    6Sep22 ARP  2.0     Chuck Nazty (Wales, Travis)  2.0   2.0   1  NaN    1   
4    6Sep22 ARP  3.0  Mucho Gifted (Aguilar, Daniel)  3.0   1.0   3  NaN    3   
5   29Aug22 ARP  5.0    Del Argent (Gonzalez, Pedro)  5.0   5.0   5  NaN    4   
6    6Sep22 ARP  1.0    Our Bubby Boy (Haar, Nathan)  1.0   3.0   2  NaN    2   
7   29Aug22 ARP  4.0     Divine Task (Herrera, Hugo)  4.0   4.0   4  NaN    5   
8    6Sep22 ARP  2.0     Chuck Nazty (Wales, Travis)  2.0   2.0   1  NaN    1   
9    6Sep22 ARP  3.0  Mucho Gifted (Aguilar, Daniel)  3.0   1.0   3  NaN    3   
10  29Aug22 ARP  5.0    Del Argent (Gonzalez, Pedro)  5.0   5.0   5  NaN    4   
11   6Sep22 ARP  1.0    Our 

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a 

     Last Raced  Pgm             Horse Name (Jockey)   PP Start 1/4  1/2  Str  \
0   12Sep22 ARP  8.0      WMA GOT IT (Martin, David)  3.0   3.0   3  1.0  1.0   
1   12Sep22 ARP  5.0       SWEET-WAN (Herrera, Hugo)  1.0   2.0   2  2.0  2.0   
2   12Sep22 ARP  7.0         CIRCE JPH (Lopez, Jose)  2.0   1.0   1  3.0  3.0   
3    6Sep22 ARP  2.0     Chuck Nazty (Wales, Travis)  2.0   2.0   1  NaN    1   
4    6Sep22 ARP  3.0  Mucho Gifted (Aguilar, Daniel)  3.0   1.0   3  NaN    3   
5   29Aug22 ARP  5.0    Del Argent (Gonzalez, Pedro)  5.0   5.0   5  NaN    4   
6    6Sep22 ARP  1.0    Our Bubby Boy (Haar, Nathan)  1.0   3.0   2  NaN    2   
7   29Aug22 ARP  4.0     Divine Task (Herrera, Hugo)  4.0   4.0   4  NaN    5   
8    6Sep22 ARP  2.0     Chuck Nazty (Wales, Travis)  2.0   2.0   1  NaN    1   
9    6Sep22 ARP  3.0  Mucho Gifted (Aguilar, Daniel)  3.0   1.0   3  NaN    3   
10  29Aug22 ARP  5.0    Del Argent (Gonzalez, Pedro)  5.0   5.0   5  NaN    4   
11   6Sep22 ARP  1.0    Our 

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


IndexError: list index out of range

In [ ]:
master_df

In [18]:
import json
with open('race_types.json') as json_file:
    race_types = json.load(json_file)

In [38]:
mc = race_types["race_types"]["maiden claiming"]["word_list"]
msp = race_types["race_types"]["maiden special weight"]["word_list"]

In [48]:
type(mc[3][1])

list

In [46]:
type(msp[3][0])

str

In [51]:
print(type(mc[3][0]) == list)

False
